# tweetfeed

# Data

In [1]:
import os
import pandas as pd
import sys
import re
from urllib.parse import urlparse
import numpy as np


In [2]:
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path + "/tweetfeed")
    
from data import load_tweets,find_news
from twitter_utils import get_collection_list, get_collection_id

In [3]:
import tweepy
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(wait_on_rate_limit=True)

class Tweet(object):
    def __init__(self, embed_str=None):
        self.embed_str = embed_str

    def _repr_html_(self):
        return self.embed_str
    
def embed_str(idx):
    return api.get_oembed(idx)["html"]

In [4]:
# df = load_tweets("../home.db", days=24)
# df.to_pickle("tweetfeed.pkl")

In [5]:
df = pd.read_pickle("tweetfeed.pkl"); df.shape

(55485, 9)

In [6]:
df.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
0,5596520,713143,soo hungry need to find my wife and head to pf...,2007-02-20T02:43:19+00:00,en,N/A,N/A,0,N/A
1,154674522,7193842,Hofstadter’s Law: A task always takes longer t...,2007-07-17T20:45:46+00:00,en,N/A,N/A,0,N/A
2,412145582,4829901,Adobe Updater must update itself before instal...,2007-11-13T21:45:35+00:00,en,N/A,N/A,0,N/A
3,922321981,16298441,no,2008-09-15T17:25:20+00:00,und,N/A,N/A,0,N/A
4,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,N/A,N/A,0,N/A


In [11]:
df = df.rename({"full_text" : "full_text_short"}, axis=1)

In [13]:
df[df.in_reply_to_status_id != "N/A"]

,id,user,full_text_short,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
47,667817009363611648,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,2015-11-20T21:29:17+00:00,und,N/A,N/A,0,667816920742129664
48,667817093123801088,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,2015-11-20T21:29:37+00:00,und,N/A,N/A,0,667817009363611648
62,709737027743440896,93278929,@ArchMaher #مسجد في مدينة #جدة #تصميم #معماري ...,2016-03-15T13:44:28+00:00,und,N/A,N/A,0,709724587823652864
68,751021335480905730,3610845615,@keffiesuplex you need to find yourself a weal...,2016-07-07T11:53:34+00:00,en,N/A,N/A,0,751014338895417344
71,755026645069750272,381289719,1) When ISIS was still occupying parts of Trip...,2016-07-18T13:09:14+00:00,en,N/A,N/A,0,755018269501841408
...,...,...,...,...,...,...,...,...,...
55473,1351678195083468800,40426297,"@collinofzion I'll take a cord of wood, thanks!",2021-01-19T23:49:29+00:00,en,N/A,N/A,0,1351544872738684929
55474,1351678756621082627,264043245,Dipert reckd,2021-01-19T23:51:43+00:00,en,N/A,N/A,0,1351678175798046723
55478,1351679374500642818,991093012804517888,@pilhuhn @mipsytipsy I got mine from https://w...,2021-01-19T23:54:10+00:00,en,N/A,N/A,0,1351572277196955651
55482,1351680121409728515,1717291,https://twitter.com/pkedrosky/status/135168012...,2021-01-19T23:57:08+00:00,und,N/A,N/A,0,1351679706379141121


In [14]:
df[df.quoted_status != "N/A"]

,id,user,full_text_short,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
85,811910511587717120,93278929,تكرّش حتى أمسى بلا رقبة #مظفر_النواب https://t...,2016-12-22T12:25:25+00:00,ar,N/A,811909512445788160,1,N/A
88,814113551778508801,3169212490,#modularity leads to #innovation @carlissbald...,2016-12-28T14:19:30+00:00,en,N/A,814110461436235776,1,N/A
124,877261616512892929,1575023431,المكان اللى اتصور فيه فيلم ذيب ❤ . كنت هموت وا...,2017-06-20T20:27:22+00:00,ar,N/A,876927696881799168,1,N/A
125,877262100606971906,93278929,@mashaly_maha https://twitter.com/archmaher/st...,2017-06-20T20:29:18+00:00,ar,N/A,667816920742129664,1,877261616512892929
200,976182659901722629,294710260,In response to this @BryanVanNorden sent me th...,2018-03-20T19:44:17+00:00,en,N/A,975771064843276288,1,N/A
...,...,...,...,...,...,...,...,...,...
55458,1351675122751569921,48008938,Dishonesty? Stupidity? Ignorance? Lack of fore...,2021-01-19T23:37:16+00:00,en,N/A,1351618345456594955,1,N/A
55460,1351675212270596097,369931522,Tying up all the loose plot lines for the fina...,2021-01-19T23:37:38+00:00,en,N/A,1351670940896284674,1,N/A
55464,1351676085667319814,738538436,Becoming. https://twitter.com/idaho_blue/statu...,2021-01-19T23:41:06+00:00,en,N/A,1351534278014713861,1,N/A
55465,1351676175794360320,17595439,After more than a century of competition and c...,2021-01-19T23:41:27+00:00,en,N/A,1351663630765469701,1,N/A


### Retweets

Retweets should be removed - tweets that are retweeted are in the database, and contain full text of a tweet, icluding links and urls. This is not a case for retweets. 

They have `RT ` in front of the tweet, and are often cut at the end.

Retweets don't add anything to consversation, they just amplify signal.

In [ ]:
# Retweet
df[df["id"] == 1340085773203193856].iloc[0].full_text

In [ ]:
# Retweeted tweet
df[df["id"] == 1339896134291091458].iloc[0].full_text

In [ ]:
# filter out retweets - retweeted tweets are in the database
# so RT only dup
df = df[df["retweeted_status"] == "N/A"] ; df.shape

## Reviewed tweets

In [ ]:
reviewed_df = pd.read_csv("../tweetfeed/data/seen.csv"); reviewed_df.shape

In [ ]:
reviewed_df["err_reason"].value_counts()

In [ ]:
deleted_tweets = reviewed_df[reviewed_df.err_reason == "not_found"].tweet_id.tolist()
df = df[~df["id"].isin(deleted_tweets)]; df.shape

In [ ]:
### let's deal only with tweet that are in English

In [ ]:
df = df[df["lang"] == "en"]; df.shape

## Augmentation

### Finding news related tweets

In [ ]:
with open("../tweetfeed/data/news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())
# df = find_news(df, news_domains); df.shape

In [ ]:
df_news = df.copy()

In [ ]:
def remove_tw_urls(tweet: str) -> str:
    """removes twitter links / urls from tweet"""
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"https://api.twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://api.twitter.com/\S+", "", tweet)
    return tweet

def rem_short_links(tweet: str) -> str:
    """removes some of short links (bit.ly, buff.ly, t.co) from tweets"""
    tweet = re.sub(r"https://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"http://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"https://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"http://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"https://t.co/\S+", "", tweet)
    tweet = re.sub(r"http://t.co/\S+", "", tweet)
    return tweet

In [ ]:
df_news["clean_text"] = df_news["full_text"].apply(remove_tw_urls).apply(rem_short_links)

In [ ]:
df_news[df_news["clean_text"].str.contains("twitter")].iloc[0].full_text

In [ ]:
df_news[df_news["clean_text"].str.contains("bit.ly")]

In [ ]:
def find_url(tweet: str) -> list:
    """find all urls in string and returns a list of all urls"""
    return re.findall(r"http\S+", tweet)

In [ ]:
df_news["urls"] = df_news["clean_text"].apply(find_url) 

In [ ]:
df_news.drop(["clean_text"], axis=1, inplace=True)

In [ ]:
def get_domain(url: str) -> str:
    """extracts domain from url, returns it"""
    domain = urlparse(url).netloc.replace("www.", "")
    dot_split = domain.split(".")
    if (len(dot_split) > 2) & (
        dot_split[-1] == "com"
    ):  # for links like "edition.cnn.com", but not like "site.co.nz"
        return ".".join(dot_split[1:])
    else:
        return domain
def remove_empty_str(string_list: list) -> list:
    """removes items that are empty strings from the list"""
    for i in string_list:
        if len(string_list) == 0:
            string_list.remove(i)
    return string_list

In [ ]:
df_news["domains"] = df_news.urls.apply(lambda x: [get_domain(d) for d in x]).apply(
    remove_empty_str
)
df_news.drop(["urls"], axis=1, inplace=True)

In [ ]:
# get max value of domains, expand each one to new column (unpack from list)
new_columns_list = []
max_nr_dom = df_news.domains.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"domain{i+1}")
df_news.reset_index(drop=True, inplace=True)
df_news[new_columns_list] = pd.DataFrame(df_news.domains.tolist())

for col in new_columns_list:
    df_news[col] = df_news[col].isin(news_domains)

In [ ]:
df_news[[
    "id", "domains", "domain1", "domain2", "domain3", "domain4", "domain5", "domain6"
]].head(10)

In [ ]:
df_news[df_news.domain1 == True][[
    "id", "domains", "domain1", "domain2", "domain3", "domain4", "domain5", "domain6"
]].head(10)

In [ ]:
df_news[df_news.domain4 == True][[
    "id", "domains", "domain1", "domain2", "domain3", "domain4", "domain5", "domain6"
]].head(10) # max domains in tweet is 6, but max *news* domains is 4.

In [ ]:
df[df["id"] == 1344730628407881729].iloc[0].full_text

In [ ]:
most_links = (
    df_news.domains.apply(lambda x: len(x)).sort_values(ascending=False).index[0]
) #get index of tweet with most links

In [ ]:
Tweet(embed_str(df_news.iloc[most_links]["id"]))

In [ ]:
df_news.drop(["domains"], axis=1, inplace=True)
# sum it all up
df_news["contains_news"] = df_news[new_columns_list].sum(axis=1)

In [ ]:
df_news["contains_news"].value_counts()

In [ ]:
# we are interest in only a True-False, so let's change that.
df_news["contains_news"] = df_news.contains_news.apply(lambda x: x if x == 0 else 1)
# now we can remove added columns
df_news.drop(new_columns_list, axis=1, inplace=True)

In [ ]:
df_news["contains_news"].value_counts()

In [ ]:
def find_news(df: pd.DataFrame, news_domains_list: list) -> pd.DataFrame:
    """Takes DataFrame, and list of domains of news sites.
    Removes from DataFrame rows that contain links to sites from that list.
    Args:
        df (pd.DataFrame): DataFrame to be cleaned
        news_domains_list (list): list of domains of news sites

    Returns:
        pd.DataFrame: DataFrame without tweets linking to news
    """
    df = df.copy()
    df["clean_text"] = (
        df["full_text"].apply(remove_tw_urls).apply(rem_short_links)
    )
    df["clean_text"] = df["clean_text"].apply(rem_short_links)
    df["urls"] = df["clean_text"].apply(find_url)
    df.drop(["clean_text"], axis=1, inplace=True)
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)

    # get max value of domains, expand each one to new column (unpack from list)
    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df.reset_index(drop=True, inplace=True)
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df.drop(["domains"], axis=1, inplace=True)
    # sum it all up
    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    # remove added columns
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

In [ ]:
df = find_news(df, news_domains); df.shape

In [ ]:
df_news["contains_news"].value_counts()

## explore muted accounts, and muted words

In [ ]:
with open("../tweetfeed/data/mute_list.txt", "r") as f:
    mute_list = json.loads(f.read())
with open("../tweetfeed/data/mute_list_cs.txt", "r") as f:
    mute_list_cs = json.loads(f.read())
with open("../tweetfeed/data/mutedacc.txt", "r") as f:
    mutedacc = json.loads(f.read())

In [ ]:
mute_list_cs

In [ ]:
mute_list[:4]

In [ ]:
mutedacc[:5]

In [ ]:
df_mute = df.copy()
m_list = mute_list
column_name = "full_text"
for item in m_list:
    df_mute[item] = df_mute[column_name].str.contains(item)
df_mute["is_mute_list"] = df_mute[mute_list].sum(axis=1)
df_mute["is_mute_list"] = df_mute["is_mute_list"].apply(lambda x: x if x == 0 else 1)
df_mute.drop(mute_list, axis=1, inplace=True)
df_mute[df_mute.is_mute_list > 0].shape

In [ ]:
def is_muted(df, mute_list, column_name, new_col, case=True):
    df_mute = df.copy()
    for item in mute_list:
        df_mute[item] = df_mute[column_name].str.contains(item, case)
    df_mute[new_col] = df_mute[mute_list].sum(axis=1)
    df_mute[new_col] = df_mute[new_col].apply(lambda x: x if x == 0 else 1)
    df_mute.drop(mute_list, axis=1, inplace=True)
    return df_mute

In [ ]:
df = is_muted(df, mute_list, "full_text", "is_mute_list", case=False)
df = is_muted(df, mute_list_cs, "full_text", "is_mute_list_cs")
df["mutedacc"] = np.where(df["user"].isin(mutedacc), 1, 0)

In [ ]:
columns_list = ["is_mute_list", "is_mute_list_cs", "mutedacc"]
df["muted"] = (
    df[columns_list]
    .sum(axis=1)
    .apply(lambda x: x if x == 0 else 1)
)
df.drop(columns_list, axis=1, inplace=True)

In [ ]:
df[["id", "user", "contains_news", "muted"]].head(10)

In [ ]:
df[(df.contains_news > 0) | (df.muted > 0)].shape #.copy and put into new dataframe

In [ ]:
reviewed_df = pd.read_csv("../tweetfeed/data/seen.csv"); reviewed_df.shape

In [ ]:
Tweet(embed_str("177008089394970624"))

In [ ]:
seen = reviewed_df[reviewed_df.err_reason == "no_errors"].tweet_id.tolist(); len(seen)

In [ ]:
df[df.id.isin(seen) & (df.contains_news > 0) & (df.muted == 0)]

In [ ]:
Tweet(embed_str("1330677864555896834"))

In [ ]:
# I remember seeing this tweet, and  then I added `tvnz.co.nz` to domains containing news

In [ ]:
auth = "../auth/auth.json"
owner_id = "143058191"

In [ ]:
not_relevant_list = get_collection_list(
    get_collection_id(owner_id, "not_relevant", auth), auth
)

In [ ]:
not_relevant_list

In [ ]:
df["not_relevant_list"] = np.where(df["id"].isin(not_relevant_list), 1, 0)

In [ ]:
df[df.not_relevant_list > 0]

In [ ]:
df[df["id"] == 1330560915616526339]

In [ ]:
# this shows, that I should do a concat of full_text of a tweet and tweets that are quoted, or replied to.
# will do that after this iteration.

## Exploratory Data Analysis (EDA)

In [ ]:
# which accounts post most news?
# which accounts post most?




## Annotation

## Labeling
## Spliting
## Preprocessing


# Modeling
## Random
## Rule-based
## Simple ML
## CNN
## RNN
## Transformers


# Experiment tracking
## Training
## Tracking
## Viewing
## Loading

# Optimization